In [ ]:
import pandas as pd
import numpy as np

In [ ]:
csv = "combined_data_MP_NE.csv"
df = pd.read_csv(csv)
df

## rename columns (ger->en)

In [ ]:
df.rename(columns={'Bucket': 'year-month'}, inplace=True)
df.rename(columns={'Wochentag': 'weekday'}, inplace=True)
df.rename(columns={'Tageszeit': 'daytime'}, inplace=True)
df.rename(columns={'Schulferien': 'isSchoolHoliday'}, inplace=True)
df.rename(columns={'Distanz': 'distance'}, inplace=True)
df.rename(columns={'Count': 'count'}, inplace=True)
df.rename(columns={'StartName': 'startName'}, inplace=True)
#df.rename(columns={'StartId': 'startID'}, inplace=True)
df.rename(columns={'EndName': 'endName'}, inplace=True)
#df.rename(columns={'EndId': 'endID'}, inplace=True)
#df.rename(columns={'StartLon': 'startLon'}, inplace=True)
#df.rename(columns={'StartLat': 'startLat'}, inplace=True)
#df.rename(columns={'EndLon': 'endLon'}, inplace=True)
#df.rename(columns={'EndLat': 'endLat'}, inplace=True)
df

In [ ]:
unique_values_StartId = df.StartId.unique()
unique_values_StartId.sort()
unique_values_StartId

## split StartId

In [ ]:
df[['startClusterID', 'startID']] = df['StartId'].str.split('_', expand=True)
df

In [ ]:
df.startID.unique()

## split StartName
before we can split Startname we have to unify the values

In [ ]:
# Function to append the string to values shorter than 29 characters
def append_string(value, cluster_id):
    if len(value) < 29:
        return value + " (30000 - " + str(cluster_id) + ")"
    else:
        return value

# Apply the function to the 'StartName' column
df['startName'] = df.apply(lambda row: append_string(row['startName'], row['startClusterID']), axis=1)
df

check if it worked

In [ ]:
unique_values_StartName = df.StartName.unique()
unique_values_StartName.sort()
unique_values_StartName

now split

In [ ]:
df[['startClusterName', 'startClusterZip']] = df['startName'].str.split(' \(3', expand=True) #extract(r'^(.*?)\s\((.*)\)$')
df['startClusterZip'] = df['startClusterZip'].astype(str).str.split('-').apply(lambda x: x[0])
df['startClusterZip'] = df['startClusterZip'].astype(int) + 30000
df

alternate way to get the startClusterID

In [ ]:
df['startClusterID'] = df['startName'].apply(lambda x: x.split('- ')[1][:-1]) #extract(r'^(.*?)\s\((.*)\)$')
df

check results

In [ ]:
unique_values_startClusterName = df.startClusterName.unique()
unique_values_startClusterName.sort()
unique_values_startClusterName

In [ ]:
unique_values_startClusterZip = df.startClusterZip.unique()
unique_values_startClusterZip.sort()
unique_values_startClusterZip

## repeat the same for EndId and EndName

### split endid

In [ ]:
df[['endClusterID', 'endID']] = df['EndId'].str.split('_', expand=True)
df

### split endname

before we can split EndName we have to unify the values

In [ ]:
# Apply the function to the 'EndName' column
df['endName'] = df.apply(lambda row: append_string(row['endName'], row['endClusterID']), axis=1)
df

check if it worked

In [ ]:
unique_values_EndName = df.EndName.unique()
unique_values_EndName.sort()
unique_values_EndName

now split

In [ ]:
df[['endClusterName', 'endClusterZip']] = df['endName'].str.split(' \(3', expand=True) #extract(r'^(.*?)\s\((.*)\)$')
df['endClusterZip'] = df['endClusterZip'].astype(str).str.split('-').apply(lambda x: x[0])
df['endClusterZip'] = df['endClusterZip'].astype(int) + 30000
df

alternate way to get endClusterID

In [ ]:
df['endClusterID'] = df['EndName'].apply(lambda x: x.split('- ')[1][:-1]) #extract(r'^(.*?)\s\((.*)\)$')
df

check result

In [ ]:
unique_values_endClusterName = df.endClusterName.unique()
unique_values_endClusterName.sort()
unique_values_endClusterName

In [ ]:
unique_values_endClusterZip = df.endClusterZip.unique()
unique_values_endClusterZip.sort()
unique_values_endClusterZip

# split bucket into year and month

In [ ]:
df[['year', 'month']] = df['year-month'].str.split('-',expand=True)
df['year'] = df['year'].astype(int)
df['month'] = df['month'].astype(int)
df

## rename weekdays

In [ ]:
df.replace({'weekday':{'Mo':'Monday','Di':'Tuesday','Mi':'Wednesday','Do':'Thursday','Fr':'Friday','Sa':'Saturday','So/Fe':'Sunday/Holiday'}}, inplace=True)
df

## convert isSchoolHoliday to boolean

In [ ]:
# Convert values of 'isSchoolHoliday' to Boolean Type using numpy.where()
df['isSchoolHoliday'] = np.where(df['isSchoolHoliday'].isin(['Ferien']), True, False)
df['isSchoolHoliday'] = df['isSchoolHoliday'].astype(bool)
df

# drop columns no longer used

In [ ]:
df = df.drop(columns=['StartId', 'EndId'])
df

In [ ]:
df = df.drop(columns=['year-month'])
df

In [ ]:
df.dtypes

In [ ]:
df.isnull().any()

# reindex columns

In [ ]:
df_result = df.reindex(columns=['year', 'month','startClusterName', 'startClusterZip', 'startClusterID', 'startID', 'endClusterName', 'endClusterZip', 'endClusterID', 'endID', 'weekday', 'daytime','isSchoolHoliday', 'distance', 'count'])
df_result

In [ ]:
df_result['startClusterID'].unique()

In [ ]:
df_result['endClusterID'].unique()

In [ ]:
df_result.dtypes

In [ ]:
df_result.fillna(0, inplace=True)

In [ ]:
df_result['startID'] = df_result['startID'].astype(int)
df_result['endID'] = df_result['endID'].astype(int)

In [ ]:
df_result.to_csv(csv.split(sep='.')[0] + '_dT' + '.csv', index=False)